# WNBA Model — Data Loader & Prep (Excel → Model Tables)

This notebook:
- Loads Excel workbook with sheets: `players`, `players_raw_stats`, `league`, `teams`, `expansion`
- Cleans numeric columns (salary commas, blanks)
- Computes `perf_score` (performance index) and `injury_risk` if missing
- Produces a final modeling table `players_model` ready for optimization/simulation


In [1]:
# 0) Set Excel file path

EXCEL_PATH = "..\data\icm_2026_wnba_data.xlsx"

import pandas as pd
import numpy as np
from pathlib import Path

path = Path(EXCEL_PATH)
if not path.exists():
    raise FileNotFoundError(f"Excel file not found: {path.resolve()}")

print("Loaded:", path.resolve())


<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\phaml\AppData\Local\Temp\ipykernel_46416\2023951944.py:3: SyntaxWarning: invalid escape sequence '\d'
  EXCEL_PATH = "..\data\icm_2026_wnba_data.xlsx"


Loaded: C:\Users\phaml\Documents\ICM-2026\data\icm_2026_wnba_data.xlsx


In [2]:
# === 1) Load sheets ===
players = pd.read_excel(EXCEL_PATH, sheet_name="players")
raw = pd.read_excel(EXCEL_PATH, sheet_name="player raw stats")
league = pd.read_excel(EXCEL_PATH, sheet_name="league")
teams = pd.read_excel(EXCEL_PATH, sheet_name="teams")

display(players.head())
display(raw.head())
display(league)
display(teams)


,player_name,team,salary,perf_score,injury_risk,position,age,social_media_followers,popularity_score
0,Kelsey Mitchell,Fever,269244,NaN,NaN,G,29,122000,NaN
1,Natasha Howard,Fever,214466,NaN,NaN,F,33,73100,NaN
2,Sophie Cunningham,Fever,100000,NaN,NaN,G/F,28,1300000,NaN
3,Damiris Dantas,Fever,100000,NaN,NaN,F/C,32,41200,NaN
4,Sydney Colson,Fever,90000,NaN,NaN,G,34,364000,NaN


,player_name,points_per_game,rebounds_per_game,assists_per_game,steals_per_game,blocks_per_game,turnovers_per_game,minutes_per_game,games_played
0,Kelsey Mitchell,20.2,1.8,3.4,0.9,0.2,1.8,31.4,44.0
1,Caitlin Clark,16.5,5.0,8.8,1.6,0.5,5.1,31.1,13.0
2,Aliyah Boston,15.0,8.2,3.7,1.2,0.9,1.8,30.2,44.0
3,Natasha Howard,11.4,6.6,1.5,1.2,0.6,1.9,24.1,44.0
4,Odyssey Sims,10.3,1.8,4.0,0.6,0.0,1.3,26.4,12.0


,season_year,salary_cap,min_roster_size,max_roster_size,total_games,team_minimum_salary,player_max_salary,supermax_salary
0,2025,1507100,11,12,44,1261440,214466,249244


,team_name,city,arena_capacity,avg_home_attendance,home_games,market_size,lat,lon,win_pct,home_wins,home_losses
0,Indiana Fever,Indianapolis,18274,16560.23,22,2080000,39.763889,-86.155556,0.545,13,9


In [3]:
# 2) Clean numeric columns (salary, stats)

def to_number(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x)
    # remove $ and commas
    s = s.replace("$", "").replace(",", "").strip()
    if s == "":
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

num_cols_players = ["salary", "perf_score", "injury_risk", "popularity_score", "age"]
for c in num_cols_players:
    if c in players.columns:
        players[c] = players[c].apply(to_number)

num_cols_raw = [
    "points_per_game","rebounds_per_game","assists_per_game","steals_per_game",
    "blocks_per_game","turnovers_per_game","minutes_per_game","games_played"
]
for c in num_cols_raw:
    if c in raw.columns:
        raw[c] = raw[c].apply(to_number)

league_cols = ["season_year","salary_cap","min_roster_size","max_roster_size","total_games"]
for c in league_cols:
    if c in league.columns:
        league[c] = league[c].apply(to_number)

display(players.head())
display(raw.head())
display(league)


,player_name,team,salary,perf_score,injury_risk,position,age,social_media_followers,popularity_score
0,Kelsey Mitchell,Fever,269244.0,NaN,NaN,G,29.0,122000,NaN
1,Natasha Howard,Fever,214466.0,NaN,NaN,F,33.0,73100,NaN
2,Sophie Cunningham,Fever,100000.0,NaN,NaN,G/F,28.0,1300000,NaN
3,Damiris Dantas,Fever,100000.0,NaN,NaN,F/C,32.0,41200,NaN
4,Sydney Colson,Fever,90000.0,NaN,NaN,G,34.0,364000,NaN


,player_name,points_per_game,rebounds_per_game,assists_per_game,steals_per_game,blocks_per_game,turnovers_per_game,minutes_per_game,games_played
0,Kelsey Mitchell,20.2,1.8,3.4,0.9,0.2,1.8,31.4,44.0
1,Caitlin Clark,16.5,5.0,8.8,1.6,0.5,5.1,31.1,13.0
2,Aliyah Boston,15.0,8.2,3.7,1.2,0.9,1.8,30.2,44.0
3,Natasha Howard,11.4,6.6,1.5,1.2,0.6,1.9,24.1,44.0
4,Odyssey Sims,10.3,1.8,4.0,0.6,0.0,1.3,26.4,12.0


,season_year,salary_cap,min_roster_size,max_roster_size,total_games,team_minimum_salary,player_max_salary,supermax_salary
0,2025.0,1507100.0,11.0,12.0,44.0,1261440,214466,249244


In [4]:
# 3) Compute performance_index (perf_score) from raw stats if missing
# perf_score = PTS + 1.2*REB + 1.5*AST + 3*(STL+BLK) - 2*TOV

raw_calc = raw.copy()
raw_calc["perf_score_calc"] = (
    raw_calc["points_per_game"]
    + 1.2*raw_calc["rebounds_per_game"]
    + 1.5*raw_calc["assists_per_game"]
    + 3.0*(raw_calc["steals_per_game"] + raw_calc["blocks_per_game"])
    - 2.0*raw_calc["turnovers_per_game"]
)

display(raw_calc[["player_name","perf_score_calc"]].head(10))


,player_name,perf_score_calc
0,Kelsey Mitchell,27.16
1,Caitlin Clark,31.80
2,Aliyah Boston,33.09
3,Natasha Howard,23.17
4,Odyssey Sims,17.66
5,Aari McDonald,19.63
6,Aerial Powers,18.32
7,Sophie Cunningham,15.70
8,Lexie Hull,17.46
9,DeWanna Bonner,15.76


In [12]:
# 4) Compute injury_risk and popularity_score
# Total games from league sheet (fallback = 40 if missing)
TOTAL_GAMES = (
    int(league.loc[0, "total_games"])
    if (
        len(league) > 0 
        and "total_games" in league.columns 
        and not pd.isna(league.loc[0, "total_games"])
    )
    else 40
)

# Injury risk
raw_calc["injury_risk_calc"] = (TOTAL_GAMES - raw_calc["games_played"]) / TOTAL_GAMES


# Preview results
display(
    raw_calc[["player_name", "games_played", "injury_risk_calc", ]].head(10)
)

print("Using TOTAL_GAMES =", TOTAL_GAMES)


,player_name,games_played,injury_risk_calc
0,Kelsey Mitchell,44.0,0.000000
1,Caitlin Clark,13.0,0.704545
2,Aliyah Boston,44.0,0.000000
3,Natasha Howard,44.0,0.000000
4,Odyssey Sims,12.0,0.727273
5,Aari McDonald,20.0,0.545455
6,Aerial Powers,8.0,0.818182
7,Sophie Cunningham,30.0,0.318182
8,Lexie Hull,44.0,0.000000
9,DeWanna Bonner,9.0,0.795455


Using TOTAL_GAMES = 44


In [ ]:
import numpy as np
import pandas as pd

# Ensure player_name matches cleanly
players["player_name"] = players["player_name"].astype(str).str.strip()
raw_calc["player_name"] = raw_calc["player_name"].astype(str).str.strip()

# Ensure social_media_followers exists in players (YOU FILL THIS IN EXCEL)
if "social_media_followers" not in players.columns:
    players["social_media_followers"] = np.nan
    print(" Add column 'social_media_followers' to the PLAYERS sheet in Excel, then re-run this cell.")

# Clean followers (handles commas, $, blanks)
def clean_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace("$", "").replace(",", "").strip()
    if s == "":
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

players["social_media_followers"] = players["social_media_followers"].apply(clean_number)

# 4) Compute popularity score (log transform)
players["popularity_score_calc"] = np.log(1 + players["social_media_followers"])

# Merge popularity into players_model (players_model must already exist OR create it here)
# If players_model doesn't exist yet, build it:
if "players_model" not in globals():
    players_model = players.merge(
        raw_calc[["player_name", "perf_score_calc", "injury_risk_calc"]],
        on="player_name",
        how="left"
    )
    # Fill computed stats if missing
    if "perf_score" in players_model.columns:
        players_model["perf_score"] = players_model["perf_score"].fillna(players_model["perf_score_calc"])
    else:
        players_model["perf_score"] = players_model["perf_score_calc"]

    if "injury_risk" in players_model.columns:
        players_model["injury_risk"] = players_model["injury_risk"].fillna(players_model["injury_risk_calc"])
    else:
        players_model["injury_risk"] = players_model["injury_risk_calc"]

# Now overwrite/attach popularity
players_model = players_model.drop(columns=["social_media_followers", "popularity_score_calc"], errors="ignore")
players_model = players_model.merge(
    players[["player_name", "social_media_followers", "popularity_score_calc"]],
    on="player_name",
    how="left"
)

# Fill popularity_score if you have that column, otherwise create it
if "popularity_score" in players_model.columns:
    players_model["popularity_score"] = players_model["popularity_score"].fillna(players_model["popularity_score_calc"])
else:
    players_model["popularity_score"] = players_model["popularity_score_calc"]

# Cleanup helper col
players_model = players_model.drop(columns=["popularity_score_calc"], errors="ignore")

# Preview
display(players_model[["player_name", "social_media_followers", "popularity_score", "perf_score", "injury_risk"]].head(15))


,player_name,social_media_followers,popularity_score,perf_score,injury_risk
0,Kelsey Mitchell,122000.0,11.711785,27.16,0.000000
1,Natasha Howard,73100.0,11.199597,23.17,0.000000
2,Sophie Cunningham,1300000.0,14.077876,15.70,0.318182
3,Damiris Dantas,41200.0,10.626218,7.83,0.136364
4,Sydney Colson,364000.0,12.804912,6.06,0.318182
5,Lexie Hull,440000.0,12.994532,17.46,0.000000
6,Brianna Turner,31000.0,10.341775,4.88,0.386364
7,Aliyah Boston,385000.0,12.861001,33.09,0.000000
8,Caitlin Clark,3600000.0,15.096445,31.80,0.704545
9,Makayla Timpson,20900.0,9.947552,6.26,0.295455


In [19]:
# ================================
# 5) Merge raw stats into players and fill computed fields (robust)
# ================================

import numpy as np
import pandas as pd

# Clean keys
players["player_name"] = players["player_name"].astype(str).str.strip()
raw_calc["player_name"] = raw_calc["player_name"].astype(str).str.strip()

# Ensure popularity_score_calc exists in PLAYERS (followers live here)
if "social_media_followers" not in players.columns:
    players["social_media_followers"] = np.nan
    print("Add 'social_media_followers' to the PLAYERS sheet.")

# Clean followers
def clean_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace("$", "").replace(",", "").strip()
    if s == "":
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

players["social_media_followers"] = players["social_media_followers"].apply(clean_number)

# Compute popularity score in PLAYERS
players["popularity_score_calc"] = np.log(1 + players["social_media_followers"])

# Merge raw stats (perf + injury) into players
players_model = players.merge(
    raw_calc[["player_name", "perf_score_calc", "injury_risk_calc"]],
    on="player_name",
    how="left"
)

# Fill perf_score / injury_risk
if "perf_score" in players_model.columns:
    players_model["perf_score"] = players_model["perf_score"].fillna(players_model["perf_score_calc"])
else:
    players_model["perf_score"] = players_model["perf_score_calc"]

if "injury_risk" in players_model.columns:
    players_model["injury_risk"] = players_model["injury_risk"].fillna(players_model["injury_risk_calc"])
else:
    players_model["injury_risk"] = players_model["injury_risk_calc"]

# Fill popularity_score from players popularity_score_calc
if "popularity_score" in players_model.columns:
    players_model["popularity_score"] = players_model["popularity_score"].fillna(players_model["popularity_score_calc"])
else:
    players_model["popularity_score"] = players_model["popularity_score_calc"]

# Drop helper cols
players_model = players_model.drop(columns=["perf_score_calc", "injury_risk_calc", "popularity_score_calc"], errors="ignore")

display(players_model.head(15))

# Write updated players_model back into Excel
from openpyxl import load_workbook
with pd.ExcelWriter(EXCEL_PATH, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    players_model.to_excel(writer, sheet_name="players_model", index=False)

print("Updated Excel sheet: players_model")


,player_name,team,salary,perf_score,injury_risk,position,age,social_media_followers,popularity_score
0,Kelsey Mitchell,Fever,269244.0,27.16,0.000000,G,29.0,122000.0,11.711785
1,Natasha Howard,Fever,214466.0,23.17,0.000000,F,33.0,73100.0,11.199597
2,Sophie Cunningham,Fever,100000.0,15.70,0.318182,G/F,28.0,1300000.0,14.077876
3,Damiris Dantas,Fever,100000.0,7.83,0.136364,F/C,32.0,41200.0,10.626218
4,Sydney Colson,Fever,90000.0,6.06,0.318182,G,34.0,364000.0,12.804912
5,Lexie Hull,Fever,88261.0,17.46,0.000000,G/F,24.0,440000.0,12.994532
6,Brianna Turner,Fever,85000.0,4.88,0.386364,F,29.0,31000.0,10.341775
7,Aliyah Boston,Fever,83371.0,33.09,0.000000,C,23.0,385000.0,12.861001
8,Caitlin Clark,Fever,78066.0,31.80,0.704545,G,23.0,3600000.0,15.096445
9,Makayla Timpson,Fever,69267.0,6.26,0.295455,F,22.0,20900.0,9.947552


Updated Excel sheet: players_model


In [20]:
# 6) Quick validation checks
required = ["player_name","team","position","salary","perf_score","injury_risk","age"]
missing_cols = [c for c in required if c not in players_model.columns]
if missing_cols:
    print("Missing required columns in players sheet:", missing_cols)

print("\nMissing values by column:")
print(players_model[required].isna().sum())

# Salary cap check (only for Fever team)
cap = league.loc[0, "salary_cap"] if (len(league) > 0 and "salary_cap" in league.columns) else np.nan
if not pd.isna(cap):
    fever_salary = players_model.loc[players_model["team"].astype(str).str.lower().eq("fever"), "salary"].sum(min_count=1)
    print(f"\nFever salary sum (from players sheet): ${fever_salary:,.0f}")
    print(f"Salary cap (league sheet): ${cap:,.0f}")
    print(f"Cap room (cap - sum): ${cap - fever_salary:,.0f}")

# Clip injury risk to [0,1]
players_model["injury_risk"] = players_model["injury_risk"].clip(0, 1)

display(players_model.sort_values("perf_score", ascending=False).head(10))



Missing values by column:
player_name    0
team           0
position       0
salary         0
perf_score     2
injury_risk    2
age            0
dtype: int64

Fever salary sum (from players sheet): $1,443,674
Salary cap (league sheet): $1,507,100
Cap room (cap - sum): $63,426


,player_name,team,salary,perf_score,injury_risk,position,age,social_media_followers,popularity_score
7,Aliyah Boston,Fever,83371.0,33.09,0.000000,C,23.0,385000.0,12.861001
8,Caitlin Clark,Fever,78066.0,31.80,0.704545,G,23.0,3600000.0,15.096445
0,Kelsey Mitchell,Fever,269244.0,27.16,0.000000,G,29.0,122000.0,11.711785
1,Natasha Howard,Fever,214466.0,23.17,0.000000,F,33.0,73100.0,11.199597
10,Aari McDonald,Fever,52333.0,19.63,0.545455,G,26.0,70200.0,11.159118
14,Aerial Powers,Fever,3975.0,18.32,0.818182,F,30.0,159000.0,11.976666
12,Odyssey Sims,Fever,7949.0,17.66,0.727273,G,32.0,39600.0,10.586610
5,Lexie Hull,Fever,88261.0,17.46,0.000000,G/F,24.0,440000.0,12.994532
17,DeWanna Bonner,Fever,72269.0,15.76,0.795455,F,37.0,95600.0,11.467939
2,Sophie Cunningham,Fever,100000.0,15.70,0.318182,G/F,28.0,1300000.0,14.077876


## Next notebook (later)

Once your data is clean, the next notebook can:
- Build the roster optimization (binary decision variables `y_i`)
- Add dynamic decision policies (state → action rules) for leverage and pricing
- Run Monte Carlo scenarios (injury + macro + demand)
